In [5]:
# Dependencies and Setup
import pandas as pd

# File to Load 
school_data_to_load = "/Users/brennancurrie/Desktop/My_Code/My_Repos/python-pandas_project/PyCitySchools/Resources/schools_complete.csv"
student_data_to_load = "/Users/brennancurrie/Desktop/My_Code/My_Repos/python-pandas_project/PyCitySchools/Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [6]:
#DISTRICT SUMMARY
# Calculate the total number of unique schools
school_count = school_data['school_name'].nunique()

# Calculate the total number of students
student_count = len(student_data)

# Calculate the total budget
total_budget = school_data['budget'].sum()

# Calculate the average (mean) math score
average_math_score = school_data_complete['math_score'].mean()

# Calculate the average (mean) reading score
average_reading_score = school_data_complete['reading_score'].mean()

# Calculate the percentage of students who passed math (math scores greather than or equal to 70
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100

# Calculate the percentage of students who passeed reading  
passing_reading_count = school_data_complete[(school_data_complete['reading_score'] >= 70)].count()['student_name']
passing_reading_percentage = passing_reading_count / float(student_count) * 100

# Calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
                                                                                    ].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100

# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools" : [school_count], "Total Students" : [student_count], "Total Budget" : [total_budget], "Average Math Score" : [average_math_score], "Average Reading Score" : [average_reading_score], "% Passing Math" : [passing_math_percentage], "% Passing Reading" : [passing_reading_percentage], "% Overall Passing" : [overall_passing_rate]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [7]:
#SCHOOL SUMMARY
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [8]:
# Calculate the total student count within each school
per_school_counts = school_data_complete['school_name'].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"])["budget"].mean()
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores
per_school_math = school_data_complete.groupby(["school_name"])["math_score"].mean()
per_school_reading = school_data_complete.groupby(["school_name"])["reading_score"].mean()

# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete[(school_data_complete['math_score'] >=70)]

# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[(school_data_complete['reading_score'] >=70)]

# Calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]

# Calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

# Created a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({"School Type" : school_types, 
                                   "Total Students" : per_school_counts,
                                   "Total School Budget" : per_school_budget,
                                   "Per Student Budget" : per_school_capita,
                                   "Average Math Score" : per_school_math,
                                   "Average Reading Score" : per_school_reading, 
                                   "% Passing Math" : per_school_passing_math, 
                                   "% Passing Reading" : per_school_passing_reading, 
                                   "% Overall Passing" : overall_passing_rate})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [9]:
#HIGHEST PERFORMING SCHOOLS (BY PERCENT OVERALL PASSING)
# Sorting the schools by `% Overall Passing` in ascending order and display the top 5 rows.
top5passing = per_school_summary.sort_values('% Overall Passing',ascending = False)
top5passing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [10]:
#LOWEST PERFORMING SCHOOLS (BY PERCENT OVERALL PASSING)
# Sorting the schools by `% Overall Passing` in descending order and display the top 5 rows.
bottom5passing = per_school_summary.sort_values('% Overall Passing')
bottom5passing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [11]:
#MATH SCORES BY GRADE
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

# Select only the `math_score`.
ninth_grader_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({"9th" : ninth_grader_math_scores,
                                    "10th" : tenth_grader_math_scores,
                                    "11th" : eleventh_grader_math_scores,
                                    "12th" : twelfth_grader_math_scores})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

TypeError: Could not convert Blake MartinFrank MarshCharles Goodman Jr.Aaron MoyerMichael DeanRobin VegaKenneth JuarezGerald SmithDavid HernandezMatthew SinghJeremiah WhiteLisa AguilarDanielle HarrisKristin JohnsonKevin MartinAnna MyersTimothy HigginsAnthony FordNicole RoseChristopher LongMr. Michael AllenCasey JonesJoseph KellyDebra BrownJacqueline SheaSara MurphyNatalie GrantPeter MillerDavid CannonValerie MayApril FisherBradley HunterJohn WilliamsonAnthony SpencerJoseph AllenAngela WallJames FerrellRobert RodriguezDarius WilliamsBrian TrevinoAndrea GreenAustin JohnstonGuy ThomasDesiree GordonScott ClarkLinda SinghWilliam PerryTerry SantanaApril StevensAnthony TorresWayne GuerreroMonique CarterChristine LeeJay SmithRobert ReyesRichard MacdonaldJonathan TuckerChelsea KempMelissa BowmanWilliam SimpsonAmanda DavisBrandon MedinaPatricia WilliamsJeremy CharlesTyler GonzalezKristen GreenRichard WallerLaura LindseyMark AlvarezMatthew ThomasHeather GouldJacqueline SnowApril Calderon MDMichae

In [12]:
#READING SCORES BY GRADE
# Separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each. 
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

# Select only the `reading_score`.
ninth_grader_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores.mean()["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({"9th" : ninth_grader_reading_scores,
                                    "10th" : tenth_grader_reading_scores,
                                    "11th" : eleventh_grader_reading_scores,
                                    "12th" : twelfth_grader_reading_scores})

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

TypeError: Could not convert Blake MartinFrank MarshCharles Goodman Jr.Aaron MoyerMichael DeanRobin VegaKenneth JuarezGerald SmithDavid HernandezMatthew SinghJeremiah WhiteLisa AguilarDanielle HarrisKristin JohnsonKevin MartinAnna MyersTimothy HigginsAnthony FordNicole RoseChristopher LongMr. Michael AllenCasey JonesJoseph KellyDebra BrownJacqueline SheaSara MurphyNatalie GrantPeter MillerDavid CannonValerie MayApril FisherBradley HunterJohn WilliamsonAnthony SpencerJoseph AllenAngela WallJames FerrellRobert RodriguezDarius WilliamsBrian TrevinoAndrea GreenAustin JohnstonGuy ThomasDesiree GordonScott ClarkLinda SinghWilliam PerryTerry SantanaApril StevensAnthony TorresWayne GuerreroMonique CarterChristine LeeJay SmithRobert ReyesRichard MacdonaldJonathan TuckerChelsea KempMelissa BowmanWilliam SimpsonAmanda DavisBrandon MedinaPatricia WilliamsJeremy CharlesTyler GonzalezKristen GreenRichard WallerLaura LindseyMark AlvarezMatthew ThomasHeather GouldJacqueline SnowApril Calderon MDMichae

In [ ]:
#SCORES BY SPENDING
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per Student Budget'],
                                                                                spending_bins, labels=labels, 
                                                                               include_lowest=True)
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

# Assemble into DataFrame
spending_summary = pd.DataFrame({'Average Math Score' : spending_math_scores,'Average Reading Score': spending_reading_scores,
                                 '% Passing Math' : spending_passing_math,
                                '% Passing Reading' : spending_passing_reading, '% Passing Overall' : overall_passing_spending})

# Display results
spending_summary

In [ ]:
#SCORES BY SIZE
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(school_spending_df['Total Students'],
                                                                size_bins, labels=labels2, 
                                                                include_lowest=True)

# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({'Average Math Score' : size_math_scores,
                             'Average Reading Score': size_reading_scores,
                             '% Passing Math' : size_passing_math,
                             '% Passing Reading' : size_passing_reading,
                             '% Passing Overall' : size_overall_passing})

# Display results
size_summary

In [ ]:
#SCORES BY SCHOOL TYPE
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["School Type"]).mean()
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()
type_passing_math = per_school_summary.groupby(["School Type"]).mean()
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()

# Select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({'Average Math Score' : average_math_score_by_type,
                             'Average Reading Score' : average_reading_score_by_type,
                             '% Passing Math' : average_percent_passing_math_by_type,
                             '% Passing Reading' : average_percent_passing_reading_by_type,
                             '% Passing Overall' : average_percent_overall_passing_by_type})

# Display results
type_summary